# Inferential statistics II - Bootstrapping

## Introduction

In the previous frequentist mini-projects, you did frequentist calculations to perform inference from a sample of data. Such inference relies on theory largely developed from the 19th-Century onwards that is subject to certain assumptions or theoretical limits. These are fine if those assumptions hold for the particular case you're working on, and what you want to do has a known theoretical distribution (for example the mean of a sampling distribution that we looked at in the previous mini-project.)

In this mini-project, you'll use the same medical charge data you used in the frequentist inference mini-project, but this time you'll make inferences about the population using bootstrapping (ie. simulating repeated re-runs of an experiment.) If frequentism is about using assumptions and theoretical results to calculate what we expect to happen were an experiment to be run again and again and again, then bootstrapping is about using computing power to essentially re-run the sample draw again and again and again to see what actually happens.

## Prerequisites

While these exercises do not strictly depend on these concepts, we encourage you to complete the previous mini-projects before starting this one so that you can approach this assignment with a good understanding of frequentist concepts like:
* the _z_-statistic
* the _t_-statistic
* the difference and relationship between the two
* the Central Limit Theorem, its assumptions and consequences
* how to estimate the population mean and standard deviation from a sample
* the concept of a sampling distribution of a test statistic, particularly for the mean
* how to combine these concepts to calculate confidence intervals and p-values
* how those confidence intervals and p-values allow you to perform hypothesis (or A/B) tests

To complete mini-project, it's important that you first complete the bootstrap resources listed in this subunit, as they contain valuable information about how to calculate bootstrap replicates of summary statistics. Having a basic understanding of what confidence intervals and p-values are will also be helpful (we touch on them in this mini-project, but please speak to your mentor or conduct individual research if you'd like to learn more.) 

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt

## Medical charge data set

In [ ]:
med_charges = pd.read_csv('data/insurance2.csv')

In [ ]:
med_charges.head()

In the previous assignment, you used the frequentist approach to estimate the lower limit for the 95% confidence interval on the mean hospital charge. This approach relies on statistical theory that has been developed over the years and is also limited to statistics for which theoretical results on the sampling distribution exist. These results are remarkably useful and applicable much of the time and under a surprisingly wide range of conditions.

Having calculated the 95% lower confidence interval using frequentist theory in the previous exercise, you'll now use bootstrap inference to verify your calculations and check that you get consistent results without making the assumptions required before. After all, the distribution of charges really was very non-normal.

__Q:__ Use bootstrap sampling to estimate the same 95% confidence interval lower limit as before.

__A:__

In [ ]:
np.random.seed(47)
N_rep = 10000

def bootstrap_replicates_1d(data, func):
    bs_sample=np.random.choice(data, len(data))
    return func(bs_sample)
bootstrap_replicates_1d(med_charges.charges, np.mean)

bs_replicates = np.empty(N_rep)
for i in range(N_rep):
    bs_replicates[i] = bootstrap_replicates_1d(med_charges.charges, np.mean)
ci_lower = np.percentile(bs_replicates, 5)
print('The lower limit pf the 95% confidence interval is ', int(ci_lower.round()))

If you performed 10000 replicates immediately after setting the random seed to 47, you should get the value 12724 here, which compares very well with the value 12725 obtained using the _t_-distribution confidence interval previously. It is a most pleasant result to see the predictions of classical frequentist theory match with results that are now possible through the number-crunching ability of computers.

Remember, in the previous mini-projects, we saw that there are two ways of performing a _t_-test from a sample, depending on whether we can assume the groups have equal variance or not. We can actually easily test this using the bootstrap approach!

__Q:__ Calculate the 95% confidence interval for the difference between the standard deviations of insurance and non-insurance claim charges (insured - non-insured). Calculate the differences over 10000 replicates. Plot the histogram of values and mark the locations of the percentiles. State the null and alternative hypothesis and comment on whether you would retain or reject the null hypothesis in this case and why.

__A:__

Null hypothesis: There will be no significant difference between the standard deviations of medical claim charges of individuals with insurance when compared to those without insurance. 
Alternative hypothesis: There will be a statistically significant difference between the standard deviations of medical claim charges of individuals with insurance compared to those without.

In [ ]:
#insurance = with insurance
#no_insurance = without insurance
insurance = med_charges[med_charges.insuranceclaim == 1]
no_insurance = med_charges[med_charges.insuranceclaim == 0]

In [ ]:
def std_diff(x0, x1):
    std_0 = bootstrap_replicates_1d(x0, np.std)
    std_1 = bootstrap_replicates_1d(x1, np.std)
    return std_0 - std_1

In [ ]:
seed(47)
std_differences = [std_diff(insurance.charges, no_insurance.charges) for i in np.arange(10000)]


In [ ]:
std_diff_ci = np.percentile(std_differences, [2.5, 97.5])
print('The 95% confidence interval for the difference in standard deviations of medical clainm charges for \
individuals with insurance when compared to those without was: ', std_diff_ci)


In [ ]:
_ = plt.hist(std_differences, bins=30)
_ = plt.xlabel('difference in standard deviation of medical claim charge (with insurance vs without)')
_ = plt.ylabel('frequency')
_ = plt.axvline(x=6681.6, color='k', linestyle='dashed', linewidth=1)
_ = plt.axvline(x=8502.7, color='k', linestyle='dashed', linewidth=1)

plt.show()

I would reject the null hypothesis, since 95% confidence interval for the differences in standard deviations is quite large (much larger than 0 (and did not contain 0), which would be expected if the standard deviations were the same).

## Confidence interval and p-value

The confidence interval above is often a useful quantity to estimate. If we wish to limit our expected probability of making a Type I error (where we wrongly reject the null hypothesis, and it is, instead, true) to $\alpha$, the associated confidence interval is our estimate of the interval within which we expect the true population value to be found $100\times(1 - \alpha)$% of the time we do this test. In the above we performed bootstrap replicates to estimate the interval and reject the null hypothesis if this interval did not contain zero. You will sometimes see such an interval reported in the output of statistical functions.

The partner of the confidence interval is the p-value. The p-value and the confidence interval are linked through our choice of $\alpha$. The p-value tells us how likely it is, under the null hypothesis, to get an outcome at least as extreme as what was observed. If this fails to reach the level of our _pre-specified_ $\alpha$, we decide the null hypothesis is sufficiently unlikely to be true and thus reject it. To calculate this p-value via the bootstrap, we have to put ourselves in a position where we are simulating the null hypothesis being true and then calculate the fraction of times we observe a result at least as extreme as that actually observed.

Remember how, previously, you used the _t_-test to calculate the p-value for the observed difference between the means of insured and non-insured medical cases. We're now going to repeat this, this time using the bootstrap approach.

__Q:__ Perform a bootstrapped hypothesis test at the 5% significance level ($\alpha = 0.05$) to calculate the p-value of the observed difference between insurance and non-insurance charges, state your null and alternative hypotheses and whether you retain or reject the null hypothesis for the given significance level.

__A:__

Null hypothesis: The means of the insured and non-insured medical cases are equal.
    Alternative hypothesis: There is a significant difference in the means of the insured vs non-insured medical cases (at 5% significance).

In [ ]:
insurance_mean = round(np.mean(insurance.charges))
print(insurance_mean)

In [ ]:
no_insurance_shifted = no_insurance.charges - np.mean(no_insurance.charges) + insurance_mean

In [ ]:
def diff_from_insured(data, insurance_mean=insurance_mean):
    return(np.mean(data)- insurance_mean)
diff_observed = diff_from_insured(no_insurance.charges)


In [ ]:
seed(47)
def bs_replicates_1d(data, func):
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)
bs_replicates_1d(insurance.charges, np.mean)

In [ ]:
def draw_bs_replicates(data, func, size=1):
    """Draw bootstrap replicates."""
    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)
    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicates_1d(data, func)
    return bs_replicates

bs_replicates = draw_bs_replicates(no_insurance_shifted, diff_from_insured, 10000)
pvalue = np.sum(bs_replicates <= diff_observed)/10000
print('The p-value of the observed difference between medical charges of insured vs uninsured cases is ', pvalue)


This means that it is extremely unlikely that the mean medical charges of insured and uninsured are equal, since 0.0 is less than the alpha level that was set (0.05).

In [ ]:
_ = plt.hist(bs_replicates, bins=30, density=True)
_ = plt.xlabel('mean charges for insured')
_ = plt.ylabel('PDF')
plt.show()

__Q:__ To put the above result in perspective, plot the histogram of your bootstrapped differences along with lines marking the locations of the observed difference. (Why would we plot more than one line, given that we only have one observed difference?)

__A:__

__Q:__ Compare your p-value above with that obtained using the _t_-test function in the previous assignment. Do you think you would want to try to perform enough bootstrap replicates to observe a random difference as large as that we did observe?

__A:__

__Q:__ Consider the two variants of the _t_-test we performed in the previous assignment. Which one would you use now?

__A:__

__Q:__ If, instead of being asked whether the means of two groups were different, you were working with an ad-recommender team who wanted to release a new recommendation algorithm, and you were looking at click-through rate both for the current algorithm (call it A) and from trials of their new algorithm (call it B), would you perform a two-sided test as above? What would be your null and alternative hypotheses and what would be the real-world consequence of rejecting the null hypothesis?

__A:__

# Learning outcomes

You've previously applied frequentist methods to calculate confidence intervals, p-values, and perform hypothesis tests. Frequentist methods use theoretical results to calculate what we expect would happen if experiments were to be run again and again and again. Now you've seen how you can do the same things using the bootstrap approach, which does not rely on such theory, and attendant assumptions, but instead literally does run experiments again and again and again.

In these exercises, you have:
* calculated the same confidence interval lower limit as you did previously
* tested the assumption that the variances of the two groups (insured vs. non-insured) were equal - something a bit harder to do using the frequentist method because of the nature of the sampling distribution for variance
* calculated the p-value for the difference between the means of the two groups and compared with the result obtained using the previous frequentist approach

You are now well equipped to apply the bootstrap approach to a wide variety of problems. Just think about what conditions you wish to recreate in your simulated reruns.